In [16]:
import glob
gt_day_train = glob.glob('/home/xzw/code/pydatatool/pydatatool/eurocity/ECPB/data/day/labels/train/*/*.json')
gt_day_val = glob.glob('/home/xzw/code/pydatatool/pydatatool/eurocity/ECPB/data/day/labels/val/*/*.json')
gt_night_train = glob.glob('/home/xzw/code/pydatatool/pydatatool/eurocity/ECPB/data/night/labels/train/*/*.json')
gt_night_val = glob.glob('/home/xzw/code/pydatatool/pydatatool/eurocity/ECPB/data/night/labels/val/*/*.json')

In [17]:
gt = gt_day_train+gt_day_val+gt_night_train+gt_night_val

In [18]:
gt.sort()

In [19]:
import os
img_name = [ os.path.split(s)[-1] for s in gt]

In [20]:
city = [c.split('_')[0] for c in img_name]

In [21]:
city_set = set(city)

In [22]:
index = [i for i in range(31)]
city_list = list(city_set)
city_list.sort()
t = zip(city_list,index)
city_id = dict(t)

In [23]:
frame_id = [os.path.splitext(c)[0].split('_')[1] for c in img_name]

In [24]:
img_id = [ "{:0>2}{}".format(city_id[city[i]],frame_id[i]) for i in range(0,len(city))]

In [25]:
img_id_set = set(img_id)

In [26]:
gt_fns = [os.path.basename(p) for p in gt]

In [27]:
img_path = ["{}/{}_{}.png".format(c,c,f) for c,f in zip(city,frame_id)]

In [28]:
def _prepare_ecp_gt(gt):
    def translate_ecp_pose_to_image_coordinates(angle):
        angle = angle + 90.0

        # map to interval [0, 360)
        angle = angle % 360

        if angle > 180:
            # map to interval (-180, 180]
            angle -= 360.0

        return np.deg2rad(angle)

    orient = None
    if gt['identity'] == 'rider':
        if len(gt['children']) > 0:  # vehicle is annotated
            for cgt in gt['children']:
                if cgt['identity'] in ['bicycle', 'buggy', 'motorbike', 'scooter', 'tricycle',
                                       'wheelchair']:
                    orient = cgt.get('Orient', None) or cgt.get('orient', None)
    else:
        orient = gt.get('Orient', None) or gt.get('orient', None)

    if orient:
        gt['orient'] = translate_ecp_pose_to_image_coordinates(orient)
        gt.pop('Orient', None)

def get_gt_frame(gt_dict):
    if gt_dict['identity'] == 'frame':
        pass
    elif '@converter' in gt_dict:
        gt_dict = gt_dict['children'][0]['children'][0]
    elif gt_dict['identity'] == 'seqlist':
        gt_dict = gt_dict['children']['children']

    # check if json layout is corrupt
    assert gt_dict['identity'] == 'frame'
    return gt_dict

In [35]:
def get_categories():
    cat = [{'id':1, 'name':'pedestrian'},
           {'id':2, 'name':'rider'},
           {'id':3, 'name':'bicycle'},
           {'id':4, 'name':'motorbike'},
           {'id':5, 'name':'person-group-far-away'},
           {'id':6, 'name':'rider+vehicle-group-far-away'},
           {'id':7, 'name':'bicycle-group'},
           {'id':8, 'name':'buggy-group'},
           {'id':9, 'name':'motorbike-group'},
           {'id':10, 'name':'scooter-group'},
           {'id':11, 'name':'tricycle-group'},
           {'id':12, 'name':'wheelchair-group'}
           ]
    return cat
def get_category_id(name):
    id = -1
    cats = get_categories()
    for cat in cats:
        if cat['name'] == name:
            id = cat['id']
    return id

def get_category_name(id):
    name=''
    cats = get_categories()
    for cat in cats:
        if cat['id'] == id:
            name = cat['id']
    return name
def ecp_occlusion_to_coco(tags):
    val = 0
    level_tags = {
        'occluded>10':10,
        'occluded>40':40,
        'occluded>80':80
    }
    for t in tags:
        if t in level_tags:
            val = level_tags[t]/100.0
    return val
                
def ecp_truncation_to_coco(tags):
    val = 0
    for t in tags:
        if 'truncated' in t:
            for t_val in [10, 40, 80]:
                if str(t_val) in t:
                    val = t_val / 100.0
                    break
    return val

In [30]:
import json
import re
import numpy as np
def load_ecp(path,time,mode):
    gt_path = os.path.join(path,'{}/labels/{}'.format(time,mode))
    gt_ext = '.json'
    gt_files = glob.glob(gt_path + '/*/*' + gt_ext)
    gt_files.sort()
    gts_ecp = []
    images = []
    for gt_file in gt_files:
        gt_fn = os.path.basename(gt_file)
        gt_frame_id = re.search('(.*?)' + gt_ext, gt_fn).group(1)
        with open(gt_file, 'rb') as f:
            gt = json.load(f)
        gt_frame = get_gt_frame(gt)
        for gt in gt_frame['children']:
            _prepare_ecp_gt(gt)
        c_i = gt_fn.split('_')[0]
        f_i = gt_fn.split('_')[1].split('.')[0]
        im_name = "{}/{}_{}.png".format(c_i,c_i,f_i)
        id = "{:0>2}{}".format(city_id[c_i],f_i)
        image = {
            "id": id,
            "im_name": im_name,
            "height": gt_frame['imageheight'],
            "width": gt_frame['imagewidth'],
            "tags": gt_frame['tags']
        }
        images.append(image)
        gts_ecp.append(gt_frame['children'])
    return gts_ecp,images

In [31]:
def ecp_to_coco(gt_ecp,objId,imgId):
    y0 = max(0, gt['y0'])
    y1 = min(1024, gt['y1'])
    x0 = max(0, gt['x0'])
    x1 = min(1920, gt['x1'])
    h = max(0,y1-y0)
    w = max(0,x1-x0)
    anno = {}
    anno['id'] = objId
    anno['instance_id'] = objId
    anno['image_id'] = imgId
    anno['category_id'] = get_category_id(gt_ecp['identity'])
    anno['identity'] = gt_ecp['identity']
    anno['bbox'] = [x0,y0,w,h]
    anno['bbox'] = [x0,y0,w,h]
    anno['ignore'] = 0
    anno['iscrowd'] = anno['category_id']==3
    anno['occl'] = ecp_occlusion_to_coco(gt_ecp['tags'])
    anno['segmentation'] = []
    anno['area'] = w*h
    anno['trunc'] = ecp_truncation_to_coco(gt_ecp['tags'])
    anno['orient'] = gt_ecp.get('orient', None)
    anno['tags'] = gt_ecp['tags']
    return anno

In [36]:
path = '/home/xzw/code/pydatatool/pydatatool/eurocity/ECPB/data'
time = 'day'
mode = 'train'
gts_ecp,images = load_ecp(path,time,mode)
annotations = []
objId = 1
for gts,img in zip(gts_ecp,images):
    for gt in gts:
        ann = ecp_to_coco(gt,objId,img['id'])
        objId += 1
        annotations.append(ann)
gt_coco = {"categories":get_categories(),
            "images":images,
            "annotations":annotations}



In [33]:
from pydatatool.utils import save_json
import os
out_dir = '/home/xzw/code/pydatatool/output/ecpb'
save_path = os.path.join(out_dir,'ecpb_{}_{}.json'.format(mode,time))
save_json(gt_coco,save_path)

In [20]:
cat_names = [a['category_name'] for a in annotations]
cat_names_set = set(cat_names)

cat_names_set
```
{'bicycle',
 'bicycle-group',
 'buggy-group',
 'motorbike',
 'motorbike-group',
 'pedestrian',
 'person-group-far-away',
 'rider',
 'rider+vehicle-group-far-away',
 'scooter-group',
 'tricycle-group',
 'wheelchair-group'}
 ```